In [147]:
import pandas as pd
import dask.dataframe as dd



In [148]:
events = dd.read_csv("/Users/19823674/Downloads/event_data.csv", dtype={'user_phone': 'object', 'event_type': 'object', 'event_dtm': 'object', 'event_status': 'object', 'update_dtm': 'object', 'payout': 'float64', 'currency': 'object', 'product_type': 'object'})
adv = dd.read_csv("/Users/19823674/Downloads/adv_data.csv", dtype={'user_id': 'object', 'campaign_type': 'object', 'campaign_start_dtm': 'object', 'campaign_end_dtm': 'object', 'source_medium': 'object', 'utm_campaign': 'object', 'interface': 'object', 'currency': 'object', 'campaign_cost': 'float64'})
users = dd.read_csv("/Users/19823674/Downloads/users_data.csv", dtype={'user_phone': 'object', 'user_id': 'object', 'registration_dtm': 'object', 'delete_dtm': 'object'})


In [11]:
users.head()

,user_phone,user_id,registration_dtm,delete_dtm
0,79100004508,0000038524a303ed5876b823574848e0,2023-01-15 15:33:14.952938,NaN
1,79100005383,000003ef5bdfb2545c8d5ab693142e30,2022-11-21 22:43:54.418792,NaN
2,79100006444,0000056caffa28b60bf17bc69fe4366f,2022-12-23 03:00:04.088219,NaN
3,79100007221,00000a6f236f49c556e9a7b31d1b01e4,2022-12-23 03:31:50.935786,NaN
4,79100009619,00000cbd32277c99f2bd5e180a21f2b2,2023-01-25 14:31:17.234998,NaN


In [10]:
adv.head()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency,campaign_cost
0,db00f9c699edacc50090172cfaa3b43a,show,2023-09-18 19:39:25.390618,2023-09-18 19:39:25.390618,finance.rambler.ru / cpm,rebrand_megamaket_reels,app,RUB,NaN
1,88aa22bca4fa1d6da73deb4b5c5ee8c0,show,2023-04-11 06:47:33.199369,2023-04-11 06:47:33.199369,yandex / organic,adv_retarget_brandkeys,web,RUB,NaN
2,e9e12347e50732804d2f8ded00743a07,show,2023-10-28 04:35:48.329870,2023-10-28 04:35:48.329870,yandex / organic,bk_light_brandkeys,web,RUB,NaN
3,fa0cd363d1978a27a078152cd263c670,show,2023-05-28 18:25:18.694051,2023-05-28 18:25:18.694051,yandex / organic,retarget_mind_response,api,RUB,NaN
4,fa0cd363d1978a27a078152cd263c670,click,2023-05-31 18:55:55.352680,2023-05-31 18:55:55.352680,regular / sms,light_clickarrow_clickarrow,api,USD,1.34


In [7]:
events.head()

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type
0,1263769bddda6eaaf7c8b8b22812ad6a063ad234d80782...,+79140092454,show,2023-10-05 12:03:04.125806,NaN,NaN,NaN,RUB,mortgage
1,ae05865023d28ae72dd17955851263ead2d4daf11c69e0...,+79125904946,show,2023-05-16 10:16:54.554523,NaN,NaN,NaN,RUB,loan
2,9861b00e9c9a99a6fdb2e5c11beebe9581eb1dda36ddc4...,+79184556300,show,2023-12-23 22:46:44.066599,NaN,NaN,NaN,RUB,loan
3,349366af4823add107077299482f21031941fc6d4cd571...,+79119287528,show,2023-07-30 05:25:46.285514,NaN,NaN,NaN,RUB,loan
4,2be2933460f58bb2d9a311ede6bb6ee064989568ab57b8...,+79119287528,click,2023-07-30 05:25:46.285514,NaN,2023-07-30 05:42:59.014970,2.18,USD,loan


In [149]:
users['user_phone'] = users['user_phone'].str.replace('+', '')
users['user_phone'] = users['user_phone'].str.replace('(', '')
users['user_phone'] = users['user_phone'].str.replace(')', '')
users['user_phone'] = users['user_phone'].str.replace(' ', '')
users['user_phone'] = users['user_phone'].str.replace('-', '')


events['user_phone'] = events['user_phone'].str.replace('+', '')
events['user_phone'] = events['user_phone'].str.replace('(', '')
events['user_phone'] = events['user_phone'].str.replace(')', '')
events['user_phone'] = events['user_phone'].str.replace(' ', '')
events['user_phone'] = events['user_phone'].str.replace('-', '')

In [150]:
joined = dd.merge(events, users[['user_phone', 'user_id']], on='user_phone', how='inner')
final_data = dd.merge(joined, adv, on='user_id', how='inner')


In [151]:
cmpn_events = final_data[["user_id", "event_type", "event_dtm", "event_status", "product_type", "source_medium", "utm_campaign", "interface", "campaign_start_dtm", "campaign_end_dtm", "campaign_type", "payout", "campaign_cost"]]
cmpn_events.to_parquet("/Users/19823674/Downloads/result3.pqt")

In [152]:
ATRIBUTION_WINDOW = 30
CMPN_WINDOW = 21


In [153]:
cmpn_events = pd.read_parquet("/Users/19823674/Downloads/result3.pqt")


In [154]:
cmpn_events['campaign_start_dtm'] = pd.to_datetime(cmpn_events['campaign_start_dtm'], format='%Y-%m-%d %H:%M:%S.%f')
cmpn_events['campaign_end_dtm'] = pd.to_datetime(cmpn_events['campaign_end_dtm'], format='%Y-%m-%d %H:%M:%S.%f')
cmpn_events['event_dtm'] = pd.to_datetime(cmpn_events['event_dtm'], format='%Y-%m-%d %H:%M:%S.%f')

cmpn_events = cmpn_events[(cmpn_events["event_dtm"] >= cmpn_events["campaign_start_dtm"]) & (cmpn_events["event_dtm"] <= (cmpn_events["campaign_end_dtm"] + pd.DateOffset(CMPN_WINDOW)))]

In [155]:
sales = cmpn_events[(cmpn_events["event_status"] == "sale")]
sales = sales[["user_id", "product_type", "interface", "event_dtm"]]
sales = sales.rename({"event_dtm": "sale_dtm"}, axis="columns")

In [156]:
sales


,user_id,product_type,interface,sale_dtm
__null_dask_index__,,,,
2080,cc9450e76fd7a6b5e8f5bd14c0a9a4f1,loan,app,2023-06-15 21:20:43.702770
2874,ba680bab37704cf1bff4c180bda7c31a,deposits,app,2023-02-19 05:22:29.298675
5073,6b2c800a06aaa250309e2028c5613ae2,loan,web,2023-05-23 07:14:19.645019
7752,24e703a14dec52207ab4f41567ad0ea8,loan,web,2023-11-08 07:43:50.837848
8664,87f8a9bcff77b3546bda9e13edccc995,deposits,app,2023-08-25 01:48:00.178444
...,...,...,...,...
317401,97e966042c5740f7ef93df3601f3a945,deposits,web,2023-04-01 00:09:34.893639
318359,c41470c131e42ad7227907f65b6aafe3,deposits,api,2023-11-25 04:42:27.448578
320137,219abe6ac4b5ddc22c06b825b125bd71,deposits,app,2023-11-23 21:35:59.202345


In [157]:
cmpn_events = cmpn_events[cmpn_events["event_type"] == cmpn_events["campaign_type"]]


In [158]:
atr = pd.merge(sales, cmpn_events, on=["user_id", "product_type"], how='inner')
atr = atr[(atr["event_dtm"] <= atr["sale_dtm"]) & (atr["event_dtm"] >= (atr["sale_dtm"] - pd.DateOffset(ATRIBUTION_WINDOW)))]

In [159]:
atr = atr[~atr.event_status.isin(["deal", "approve"]) ]
atr

,user_id,product_type,interface_x,sale_dtm,event_type,event_dtm,event_status,source_medium,utm_campaign,interface_y,campaign_start_dtm,campaign_end_dtm,campaign_type,payout,campaign_cost
0,cc9450e76fd7a6b5e8f5bd14c0a9a4f1,loan,app,2023-06-15 21:20:43.702770,click,2023-06-15 21:20:43.702770,None,ios / mobile_app,brandkeys_light_mind,app,2023-06-02 07:54:25.593167,2023-06-02 07:54:25.593167,click,0.0,0.0
1,ba680bab37704cf1bff4c180bda7c31a,deposits,app,2023-02-19 05:22:29.298675,show,2023-02-19 05:22:29.298675,None,ru.android.bankproduct / mobile_app,title_target_market,app,2023-02-02 14:50:32.570685,2023-02-02 14:50:32.570685,show,NaN,NaN
2,6b2c800a06aaa250309e2028c5613ae2,loan,web,2023-05-23 07:14:19.645019,show,2023-05-23 07:14:19.645019,None,yandex / cpc,megamaket_target_reels,web,2023-05-13 06:43:56.328074,2023-05-13 06:43:56.328074,show,NaN,NaN
3,24e703a14dec52207ab4f41567ad0ea8,loan,web,2023-11-08 07:43:50.837848,show,2023-11-08 07:43:50.837848,None,yandex / cpc,rebrand_adv_market,web,2023-10-27 11:00:11.871121,2023-10-27 11:00:11.871121,show,NaN,NaN
4,87f8a9bcff77b3546bda9e13edccc995,deposits,app,2023-08-25 01:48:00.178444,show,2023-08-25 01:48:00.178444,None,ru.android.bankproduct / mobile_app,case_target_title,app,2023-08-20 20:41:04.554560,2023-08-20 20:41:04.554560,show,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6436,97e966042c5740f7ef93df3601f3a945,deposits,web,2023-04-01 00:09:34.893639,click,2023-04-01 00:09:34.893639,None,google / organic,rebrand_rebrand_brandkeys,web,2023-03-27 19:22:10.009280,2023-03-27 19:22:10.009280,click,0.0,0.0
6437,c41470c131e42ad7227907f65b6aafe3,deposits,api,2023-11-25 04:42:27.448578,show,2023-11-25 04:42:27.448578,None,email / email,light_adv_response,api,2023-11-09 08:09:01.860453,2023-11-09 08:09:01.860453,show,NaN,NaN
6438,219abe6ac4b5ddc22c06b825b125bd71,deposits,app,2023-11-23 21:35:59.202345,click,2023-11-23 21:35:59.202345,None,yandex / cpc,light_market_brandkeys,app,2023-11-14 22:36:11.224117,2023-11-14 22:36:11.224117,click,0.0,0.0
6439,64d53b24aa477c7036bf3dd0cc738cb3,loan,app,2023-11-16 02:13:53.112842,click,2023-11-16 02:13:53.112842,None,ios / mobile_app,brandkeys_target_mind,app,2023-11-16 01:43:36.822912,2023-11-16 01:43:36.822912,click,0.0,0.0


In [160]:
atr.to_excel("/Users/19823674/Downloads/aggregation.xlsx")